In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer



In [2]:
data_path = "Dataset_analyse.csv"
df = pd.read_csv(data_path)

df = df.dropna()
df = df.drop(["title","total_spectator","hebdo_rank","first_weekend","rating_press","rating_public","casting","visa","award","lang"], axis=1)
df

,year,director,country,duration,genre,first_day,first_week,copies,distributor,budget,classification_acteurs
0,1997,3,france,6300,thriller,74284.631579,172230,234,2,1.549450e+08,3.0
1,1999,3,france,5400,drame,75572.200000,154881,198,3,9.900000e+08,3.0
2,1994,3,france,5220,comedie,74284.631579,417021,189,3,1.549450e+08,3.0
3,1995,3,etatsunis,9900,aventure action,45088.600000,355642,302,3,7.200000e+07,3.0
4,1994,3,france,6600,thriller,74284.631579,39739,46,3,1.549450e+08,3.0
...,...,...,...,...,...,...,...,...,...,...,...
2202,2008,3,grandebretagne,7080,comedie,18126.000000,110442,128,3,3.072000e+07,3.0
2203,1993,1,etatsunis,6060,drame,74284.631579,32351,44,3,1.549450e+08,3.0
2204,1999,3,france,5520,drame,74284.631579,51749,-1,3,1.549450e+08,3.0
2205,1999,3,etatsunis,7080,comedie,74284.631579,63249,-1,3,1.549450e+08,3.0


In [3]:
X = df.drop(['first_week'], axis=1)
y = df.first_week

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42)

num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

onehotscale_pipeline = make_pipeline(OneHotEncoder(handle_unknown='ignore', sparse_output=False), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)

polyscale_pipeline = make_pipeline(PolynomialFeatures(2))

my_final_pipeline = make_pipeline(preprocessing, polyscale_pipeline)
my_final_pipeline.fit(X_train)

feature_names = my_final_pipeline.get_feature_names_out(X.columns)


In [4]:
from sklearn import set_config


set_config(transform_output="pandas")
model = make_pipeline(
    my_final_pipeline,
)
xdata = model.fit_transform(X_train, y_train)
xdata.head()

,1,categorical__country_allemagne,categorical__country_australie,categorical__country_belgique,categorical__country_bresil,categorical__country_canada,categorical__country_chine,categorical__country_danemark,categorical__country_espagne,categorical__country_etatsunis,...,numerical__copies^2,numerical__copies numerical__distributor,numerical__copies numerical__budget,numerical__copies numerical__classification_acteurs,numerical__distributor^2,numerical__distributor numerical__budget,numerical__distributor numerical__classification_acteurs,numerical__budget^2,numerical__budget numerical__classification_acteurs,numerical__classification_acteurs^2
1896,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,6.763612,7.802083,2.626964,7.802083,9.0,3.030303,9.0,1.020304,3.030303,9.0
1077,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.929012,4.166667,0.660213,4.166667,9.0,1.426061,9.0,0.225961,1.426061,9.0
454,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.176517,1.260417,0.169753,1.260417,9.0,1.212121,9.0,0.163249,1.212121,9.0
652,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.399354,1.895833,0.989057,1.895833,9.0,4.695304,9.0,2.449542,4.695304,9.0
465,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2.882921,1.697917,8.759501,3.395833,1.0,5.158970,2.0,26.614968,10.317939,4.0


In [5]:

model = make_pipeline(
    my_final_pipeline,
    Lasso(random_state=42, max_iter=100000)
)

# param_grid = {'lasso__alpha': [15]}


# model = GridSearchCV(model, param_grid,cv =5)
model.fit(X_train, y_train)
grid_score = model.score(X_train, y_train)

# best_alpha = model.best_params_['lasso__alpha']

/home/utilisateur/Documents/Projets/Brief_Popularity_Film/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.652e+13, tolerance: 3.753e+10
  model = cd_fast.enet_coordinate_descent(


In [6]:
# y_pred_train = model.predict(X_train)


# residus =  y_pred_train - y_train


# weights = np.exp(-abs(residus) / residus.std())
# model.fit(X_train, y_train, lasso__sample_weight=weights)




y_pred = model.predict(X_test)

In [7]:

r2_cleaned = r2_score(y_test, y_pred)
mse_cleaned = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle :")
# print(model.best_params_)
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")

Performance du modèle :
R2 Score: 0.7557
MAE: 142664.58
RMSE: 293494.81


In [8]:
-------------------------------------
---------------------------------------
-----------------------------------------
------------------------------------------
-------------------------------------------

SyntaxError: invalid syntax (2863386346.py, line 1)

In [ ]:
dfdf = pd.DataFrame(X_test)
dfdf


,year,director,country,duration,genre,copies,budget,classification_acteurs
574,1997,3,france,6300,comedie,488,1.549450e+08,3.0
2080,2006,3,france,7500,thriller,434,1.170000e+07,3.0
443,1996,3,france,5100,comedie,219,1.549450e+08,3.0
557,1996,3,etatsunis,7380,comedie,253,3.200000e+07,3.0
202,2009,3,france,5640,drame,125,3.960000e+07,3.0
...,...,...,...,...,...,...,...,...
65,2007,3,france,6600,comedie,513,8.700000e+06,3.0
1659,2023,3,italie,5760,comedie dramatique,245,1.910000e+08,3.0
982,2017,3,etatsunis,9060,fantasy,1027,9.192000e+08,3.0
1256,2020,1,etatsunis,9000,science fiction,678,9.540000e+07,2.0


In [ ]:
data = {
    'year': 2024,
    'director': 1,
    'country': "etatsunis",
    'duration': 5640,
    'genre' : "animation",
    'copies': 696,
    'budget': 85000000,
    'classification_acteurs': 2
}

data = {k:[v] for k,v in data.items()}

df_test = pd.DataFrame(data)
df_test

,year,director,country,duration,genre,copies,budget,classification_acteurs
0,2024,1,etatsunis,5640,animation,696,85000000,2


In [ ]:
y_pred = model.predict(df_test)

In [ ]:
r2_cleaned = r2_score(df_test, y_pred)
mse_cleaned = mean_squared_error(df_test, y_pred)
mae = mean_absolute_error(df_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle :")
# print(model.best_params_)
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")

ValueError: could not convert string to float: 'etatsunis'